# Pick the closest number game
Three players A, B, C play the following game. First, A picks a real number in $[0,1]$, then B picks a number in the same range (different from A’s choice) and finally C picks a number, also in the same range, (different from the two chosen numbers). We then pick a random number  $x\in[0,1]$ with uniform distribution. Whoever’s number is closest to $x$ wins the game. Assume that A, B and C all play optimally and their sole goal is to maximise their chances of winning. Also assume that if one of them has several optimal choices, then that player will randomly pick one of the optimal choices.

1. If A chooses 0, then what is the best choice for B?
2. What is the best choice for A?
3. Can you write a program to figure out the best choice for the first player when the game is played among four players?

The questions above can be solved analytically, resulting into:

1. $b=2/3$
2. $a=1/4$, $a=3/4$
3. $a=1/5$, $a=4/5$


Below is a code to address numerically all of the questions above.

The solution strategy of this sequential game is by **backward induction**.

In [2]:
import numpy as np
from fractions import Fraction

## Functions

In [3]:
def find_nearest_larger(value, array):
    """
    Find entry in 'array' nearest to 'value' and larger than it
    """
    array = np.asarray(array)    
    array = array[np.where(array>value)] # select only larger entries
    if array.size > 0:
        idx = (np.abs(array - value)).argmin()
        return( array[idx] )
    else:
        return(1)

def find_nearest_smaller(value, array):
    """
    Find entry in 'array' nearest to 'value' and smaller than it
    """
    array = np.asarray(array)    
    array = array[np.where(array<value)] # select only smaller entries
    if array.size > 0:        
        idx = (np.abs(array - value)).argmin()
        return( array[idx] )
    else:
        return(0)

    
def compute_payoff(positions):
    """
    Given the list of positions of all players, 
    compute payoff for each player and return it as a list
    (in the same order as the input positions)
    """
    
    # make list if input is not a list
    if not isinstance(positions, list):
        positions = [positions]    
    
    result = []
    xvalues = list(positions)
    
    if 0.0 not in xvalues:
        xvalues.insert(0, 0.0) # insert 0.0 at beginning
    if 1.0 not in xvalues:
        xvalues.append(1.0)   # insert 1 at end

    for x in positions:
        x_larger  = find_nearest_larger(x, xvalues)
        x_smaller = find_nearest_smaller(x, xvalues)        
        
        if( x_larger == 1.0 and 1.0 not in positions ):
            payoff_larger = 1-x        
        else:
            payoff_larger = (x_larger - x)*0.5

        if( x_smaller == 0.0 and 0.0 not in positions ):
            payoff_smaller = x            
        else:
            payoff_smaller = (x-x_smaller)*0.5

        payoff = payoff_larger + payoff_smaller        
        result.append(payoff)
    
    return(result)

def optimal_move(player, all_moves):
    """
    Select best move (maximizing payoff) for player 'player' among all possible moves.
    If more than one optimal moves are available, randomly pick one of them.
    """
    if all_moves.size > 0:
        best_move = all_moves[np.random.choice( np.argwhere(all_moves[:,1,player] 
                                               == all_moves[:,1,player].max()).flatten())]  
        best_move = np.reshape(best_move,(1,all_moves.shape[1],all_moves.shape[2]))
        return(best_move)
    else:
        return(None)

## 1. Game with 3 players A,B,C and $a=0$

In [49]:
# Discretize interval [0,1] with 1/N spacing
N = 100
space = np.array([float(Fraction(i, N)) for i in range(0, N+1)])

In [50]:
def fill_payoff_matrix_3players_1fixed(space):
    """
    Fill array [ [player_values], [player_payoffs] ]
    with all payoffs associated to each player choice
    space: discretized interval [0,1]
    """
    players = [[a,b,c] 
            for a in np.array([0.0])    # fixed choice a = 0
            for b in np.delete(space,np.where(space==a)) # avoid b=a
            for c in np.delete(space,np.where(np.logical_or(space==a,space==b))) # avoid c=a,b
          ]
    payoffs = []
    for p in players:
        payoffs.append([p, compute_payoff(p)]) # [ [player_values], [player_payoffs] ]

    payoffs = np.array(payoffs)
    return(payoffs)

In [51]:
payoffs = fill_payoff_matrix_3players_1fixed(space)

In [53]:
nplayers = payoffs.shape[-1] # 2

player=1

# Best C moves
selected_moves_c = np.zeros((0,2,nplayers)) # initialize selected nodes for C

for bb in space:
    position = payoffs[np.where(payoffs[:,0,player]==bb)]
    best_move = optimal_move(player+1, position)
    if best_move is not None:        
        selected_moves_c = np.append( selected_moves_c, best_move, axis=0 )

print("Optimal positions (a=0 fixed):")

# Best B move
for p in range(nplayers):
    print("   player {:s} = {:.3f}    (probability = {:.3f})".format(
        chr(p+ord('A')),        
        selected_moves_c[selected_moves_c[:,1,player].argmax()][0, p], 
        selected_moves_c[selected_moves_c[:,1,player].argmax()][1, p]) )

Optimal positions (a=0 fixed):
   player A = 0.000    (probability = 0.035)
   player B = 0.670    (probability = 0.630)
   player C = 0.070    (probability = 0.335)


## 2. Game with 3 players A,B,C and $a\neq 0$

In [54]:
# Discretize interval [0,1] with 1/N spacing
N = 100
space = np.array([float(Fraction(i, N)) for i in range(0, N+1)])

In [55]:
def fill_payoff_matrix_3players(space):
    """
    Fill array [ [player_values], [player_payoffs] ]
    with all payoffs associated to each player choice
    space: discretized interval [0,1]
    """    
    players = [[a,b,c] 
                for a in space[np.where(space<=0.5)] # a in [0,0.5] (by symmetry)
                for b in np.delete(space,np.where(space==a)) # avoid b=a
                for c in np.delete(space,np.where(np.logical_or(space==a,space==b))) # avoid c=a,b
              ]
    
    payoffs = []
    for p in players:
        payoffs.append([p, compute_payoff(p)]) # [ [player_values], [player_payoffs] ]

    payoffs = np.array(payoffs)
    return(payoffs)

In [56]:
payoffs = fill_payoff_matrix_3players(space)

In [57]:
nplayers = payoffs.shape[-1] # 3

player = 0 # 0=A, 1=B, 2=C

# Best C moves
selected_moves_c = np.zeros((0,2,nplayers)) # initialize selected nodes for C
for aa in space:
    for bb in space:
        position = payoffs[np.where(np.logical_and(payoffs[:,0,player]==aa,
                                                   payoffs[:,0,player+1]==bb))]
        best_move = optimal_move(player+2, position)
        if best_move is not None:
            selected_moves_c = np.append( selected_moves_c, best_move, axis=0 )

# Best B moves
selected_moves_b = np.zeros((0,2,nplayers)) # initialize selected nodes for B
for aa in space:
    position = selected_moves_c[np.where(selected_moves_c[:,0,player]==aa)]
    best_move = optimal_move(player+1, position)
    if best_move is not None:
        selected_moves_b = np.append( selected_moves_b, best_move, axis=0 )

print("Optimal positions:")

# Best A moves
for p in range(nplayers):
    print("   player {:s} = {:.3f}    (probability = {:.3f})".format(
        chr(p+ord('A')),        
        selected_moves_b[selected_moves_b[:,1,player].argmax()][0, p], 
        selected_moves_b[selected_moves_b[:,1,player].argmax()][1, p]) )

Optimal positions:
   player A = 0.230    (probability = 0.295)
   player B = 0.740    (probability = 0.450)
   player C = 0.360    (probability = 0.255)


## 3. Game with 4 players A,B,C,D 

In [42]:
# Discretize interval [0,1] with 1/N spacing
N = 40
space = np.array([float(Fraction(i, N)) for i in range(0, N+1)])

In [44]:
def fill_payoff_matrix_4players(space):
    """
    Fill array [ [player_values], [player_payoffs] ]
    with all payoffs associated to each player choice
    space: discretized interval [0,1]
    """    
    players = [[a,b,c,d] 
            for a in space[np.where(space<=0.5)] # a in [0,0.5] (by symmetry)
            for b in np.delete(space,np.where(space==a)) # avoid b=a
            for c in np.delete(space,np.where(np.logical_or(space==a,
                                                            space==b))) # avoid c=a,b
            for d in np.delete(space,np.where(np.logical_or(space==a,
                                                            np.logical_or(space==b,
                                                                          space==c)))) # avoid d=a,b,c
          ]
    payoffs = []
    for p in players:
        payoffs.append([p, compute_payoff(p)]) # [ [player_values], [player_payoffs] ]

    payoffs = np.array(payoffs)
    return(payoffs)

In [45]:
payoffs = fill_payoff_matrix_4players(space)

In [46]:
nplayers = payoffs.shape[-1] # 4

player = 0 

# Best D moves
selected_moves_d = np.zeros((0,2,nplayers)) # initialize selected nodes for D
for aa in space:
    for bb in space:
        for cc in space:
            position = payoffs[np.where(np.logical_and(payoffs[:,0,player]==aa,
                                                       payoffs[:,0,player+1]==bb, 
                                                       payoffs[:,0,player+2]==cc))]            
            best_move = optimal_move(player+3, position)
            if best_move is not None:
                selected_moves_d = np.append( selected_moves_d, best_move, axis=0 )
            
# Best C moves
selected_moves_c = np.zeros((0,2,nplayers)) # initialize selected nodes for C
for aa in space:
    for bb in space:
        position = selected_moves_d[np.where(np.logical_and(
                                                        selected_moves_d[:,0,player]==aa,
                                                        selected_moves_d[:,0,player+1]==bb))]
        best_move = optimal_move(player+2, position)
        if best_move is not None:
            selected_moves_c = np.append( selected_moves_c, best_move, axis=0 )

# Best B moves
selected_moves_b = np.zeros((0,2,nplayers)) # initialize selected nodes for B
for aa in space:
    position = selected_moves_c[np.where(selected_moves_c[:,0,player]==aa)]
    best_move = optimal_move(player+1, position)
    if best_move is not None:
        selected_moves_b = np.append( selected_moves_b, best_move, axis=0 )

print("Optimal positions:")

# Best A moves
for p in range(nplayers):
    print("   player {:s} = {:.3f}    (probability = {:.3f})".format(
        chr(p+ord('A')),        
        selected_moves_b[selected_moves_b[:,1,player].argmax()][0, p], 
        selected_moves_b[selected_moves_b[:,1,player].argmax()][1, p]) )

Optimal positions:
   player A = 0.17    (probability = 0.20)
   player B = 0.82    (probability = 0.31)
   player C = 0.85    (probability = 0.16)
   player D = 0.23    (probability = 0.33)


In [47]:
for p in range(nplayers):
    print("   player {:s} = {:.3f}    (probability = {:.3f})".format(
        chr(p+ord('A')),        
        selected_moves_b[selected_moves_b[:,1,player].argmax()][0, p], 
        selected_moves_b[selected_moves_b[:,1,player].argmax()][1, p]) )

   player A = 0.175    (probability = 0.200)
   player B = 0.825    (probability = 0.312)
   player C = 0.850    (probability = 0.163)
   player D = 0.225    (probability = 0.325)
